<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#load-dataset" data-toc-modified-id="load-dataset-1">load dataset</a></span></li><li><span><a href="#train-model" data-toc-modified-id="train-model-2">train model</a></span></li></ul></div>

# load dataset

In [1]:
import os
import tensorflow as tf
from functools import partial
from tensorflow.keras.applications import EfficientNetB0

In [2]:
def _parse_function(tfrecord_serialized, image_size):
    features = {'image': tf.io.FixedLenFeature([], tf.string),
                'label': tf.io.FixedLenFeature([], tf.int64)
               }
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)

    image = tf.io.decode_raw(parsed_features['image'], tf.uint8)
    image = tf.reshape(image, [256, 256, 3])
    image = tf.image.resize(image, [image_size, image_size])

    label = tf.cast(parsed_features['label'], tf.int64)
    label = tf.one_hot(label, 10)

    return image, label

In [3]:
tfr_path = '../data/place_10_5000.tfr'
dataset = tf.data.TFRecordDataset(tfr_path)

In [4]:
dataset_size = len(list(dataset))
train_size = int(0.8 * dataset_size)
val_size = int(0.2 * dataset_size)
batch_size = 32

In [5]:
parsed_dataset = dataset.map(
    partial(_parse_function, image_size=224), 
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)
dataset = parsed_dataset.shuffle(dataset_size)

In [6]:
train_ds = dataset.take(train_size)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.repeat()
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
val_ds = dataset.skip(train_size)
val_ds = val_ds.batch(batch_size)

# train model

In [8]:
base_model = EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

In [9]:
base_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
_____________________________________________________________________________________

In [10]:
input_layer = tf.keras.layers.Input((224, 224, 3))
model = base_model(input_layer)
model = tf.keras.layers.Dense(512, activation='relu')(model)
model = tf.keras.layers.BatchNormalization()(model)
model = tf.keras.layers.Dense(10)(model)
model = tf.keras.Model(input_layer, model)

In [11]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 4,712,621
Trainable params: 4,669,574
Non-trainable params: 43,047
_________________________________________________________________


In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [13]:
save_dir = '../models/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [14]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
mc = tf.keras.callbacks.ModelCheckpoint(
    filepath=save_dir+'{epoch}-{val_loss:.2f}-{val_accuracy:.2f}.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

In [15]:
history = model.fit(
    train_ds,
    epochs=9999,
    validation_data=val_ds,
    steps_per_epoch=dataset_size//batch_size,
    callbacks=[es, mc]
)

Epoch 1/9999
156/156 [==============================] - ETA: 0s - loss: 0.5961 - accuracy: 0.8235
Epoch 00001: val_accuracy improved from -inf to 0.93400, saving model to ../models/1-0.22-0.93.h5
156/156 [==============================] - 76s 489ms/step - loss: 0.5961 - accuracy: 0.8235 - val_loss: 0.2221 - val_accuracy: 0.9340
Epoch 2/9999
156/156 [==============================] - ETA: 0s - loss: 0.2489 - accuracy: 0.9181
Epoch 00002: val_accuracy improved from 0.93400 to 0.97100, saving model to ../models/2-0.09-0.97.h5
156/156 [==============================] - 42s 269ms/step - loss: 0.2489 - accuracy: 0.9181 - val_loss: 0.0912 - val_accuracy: 0.9710
Epoch 3/9999
156/156 [==============================] - ETA: 0s - loss: 0.1618 - accuracy: 0.9457
Epoch 00003: val_accuracy improved from 0.97100 to 0.98300, saving model to ../models/3-0.06-0.98.h5
156/156 [==============================] - 37s 238ms/step - loss: 0.1618 - accuracy: 0.9457 - val_loss: 0.0573 - val_accuracy: 0.9830
Epoc